In [19]:
import pandas as pd
import numpy as np
from groq import Groq
from openai import OpenAI
from pyairtable import Table,Api
from pprint import pprint
import networkx as nx
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.cluster.util import cosine_distance


In [20]:
api = Api('patkECjZGaeJ3Rj2F.902a4dc39c614cbcba594a1d909db35a39dcd69f43d2bf692aaf32268c4d60d7')
tableMessages = api.table('appY85YsYveVhA1eC', 'Messages')
data=tableMessages.all()
fields = [record['fields'] for record in data]
df=pd.DataFrame(fields)

In [21]:
dff = df['other'].dropna().to_list()

In [ ]:
print(dff)

In [ ]:
pprint(dff)

In [ ]:
%pip install transformers sentence-transformers

In [10]:
from networkx.algorithms.link_analysis.pagerank_alg import pagerank
from sentence_transformers import SentenceTransformer, util

/Users/neralbcika/anaconda3/envs/stage-ai/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [11]:
class BERTSummarizer:

    def __init__(self, pretrained_model='all-MiniLM-L6-v2'):
        self.bert_model = SentenceTransformer(pretrained_model)

    def create_graph(self, sentences, sentence_vectors):
        G = nx.Graph()
        for i, s1 in enumerate(sentences):
            for j, s2 in enumerate(sentences):
                # do not compute similarity with itself
                if s1 != s2:
                    similarity = util.pytorch_cos_sim(sentence_vectors[i], sentence_vectors[j])
                    G.add_edge(i, j, weight=similarity)
        return G

    def summarize(self, sentences, N=2, return_list=False):
        sentence_vectors = self.bert_model.encode(sentences)
        G = self.create_graph(sentences, sentence_vectors)
        try:
            pr_scores = pagerank(G, max_iter=1000)
        except Exception as e:
            print("The PageRank algorithm failed to converge. Returning the top sentences according to their position in the text.")
            pr_scores = {i: N-i for i in range(len(sentences))}

        ordered_pr = dict(sorted(pr_scores.items(), key=lambda item: item[1], reverse=True))
        ordered_keys = list(ordered_pr.keys())
        summary_sents = []
        for ind in ordered_keys[:N]:
            summary_sents.append(sentences[ind])
        if return_list:
            return summary_sents
        else:
            return " ".join(summary_sents)

In [17]:
bs = BERTSummarizer()
summary = bs.summarize(dff)
print (summary)

The PageRank algorithm failed to converge. Returning the top sentences according to their position in the text.
non ci interessa lavorare specificatamente sulle loro tecnologie non chiederei se sta bene nel primo messaggio, probabilmente troppo lungo


In [31]:
phrases="\n".join(dff)
num_sent=5
model = "mixtral-8x7b-32768"
client = Groq(
        api_key='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',
    )

context = f"""You are given a list of phrases in Italian. Your task is to understand the semantic meaning of these phrases and generate sentences that best describe the essence of the phrases.
Act like you are the person writing those phrases.
Make them short and understandable, capturing key aspects. 
List {num_sent} of the most important objections made.
Here are the phrases:\n\n{phrases}"""

messages = [
    {"role": "system", "content": "You are an AI language model."},
    {"role": "user", "content": context}
]

chat_completion = client.chat.completions.create(
                    messages=messages,
                    model=model,
                    max_tokens=32600,
                    temperature=1
                )
llm_response = chat_completion.choices[0].message.content
pprint(llm_response)


('Here are five key objections synthesized from the given Italian phrases:\n'
 '\n'
 '1. The focus on specific technologies or roles is not of interest. The '
 'messages should avoid emphasizing on certain technologies or positions, as '
 'Airbag Studio is a fully digital company and the role of the person being '
 'addressed is not relevant. The message length should also be reduced, and '
 'errors due to translation should be corrected.\n'
 '\n'
 '2. The messages appear to be overly formal or insincere, containing phrases '
 'like "sono emozionato di conoscerti" which would be more appropriate for '
 'well-known personalities. Also, using the name of the company, Volcanic '
 'Minds, should be consistent throughout the message.\n'
 '\n'
 '3. Incorrect references or assumptions have been made. The founder of Airbag '
 'Studio has not been made aware of any plans or strategies. Additionally, '
 'some mentioned sectors, like "mobile senior consultant" or "settore buildo," '
 "do not exis